# DOCTOR-JI : Predicting Name of Drug for  Disease

---



Importing Thirdai

In [ ]:
!pip3 install thirdai --upgrade
import thirdai


##Activating License and api key

In [ ]:
thirdai.licensing.deactivate()
thirdai.licensing.activate("1FB7DD-CAC3EC-832A67-84208D-C4E39E-V3")


# Importing dataset
---



In [ ]:
import pandas as pd
import os

# os.system('wget -O amazon-kaggle-product-catalog.csv https://www.dropbox.com/s/kq5396ypmtagsyr/amzn-kaggle-product-catalog.csv?dl=0')

# catalog_file = "./amazon-kaggle-product-catalog.csv"

catalog_file="/content/drive/MyDrive/ThirdAI_Hackathon/Main Project Files/drugs_for_common_treatments.csv"
## for a quick demo, we are sampling just the first 5% of the products
def sample_catalog(catalog_file, percentage=0):
    df = pd.read_csv(catalog_file)
    df["alcohol"]=df["alcohol"].fillna(0)
    df["alcohol"]=df["alcohol"].replace("X",1)

    df = df.sample(frac=percentage, random_state=2)                #Try to Use random state 10,20,30,40 for finding changes in output -- aa dhyan rakhjo
    df["DOC_ID"] = [i for i in range(df.shape[0])]
    df["drug_name"] = df["drug_name"].str.lower()
    df["rating"]=df["rating"].fillna(df.rating.mean())
    df.rating=df.rating.astype("str")
    df.alcohol=df.alcohol.astype("str")
    df["medical_condition"] = df["medical_condition"].str.lower()
    df["medical_condition_description"] = df["medical_condition_description"].str.lower()
    df["rx_otc"]=df["rx_otc"].str.lower()
    df["rating"]=df["rating"].str.lower()
    df["alcohol"]=df["alcohol"].str.lower()
    #
    sampled_catalog_file = f"/content/drive/MyDrive/ThirdAI_Hackathon/Main Project Files/drugs_for_common_treatments-{percentage}.csv"
    df.to_csv(sampled_catalog_file, index=False)
    #
    return sampled_catalog_file, df

sampled_catalog_file, dataframe = sample_catalog(catalog_file, 0.75)

In [ ]:
## A sample row from the sampled catalog file is printed below.
pd.options.display.max_colwidth = 700
dataframe[dataframe["DOC_ID"] == 3]


,DOC_ID,drug_name,medical_condition,medical_condition_description,activity,rx_otc,pregnancy_category,csa,alcohol,rating,no_of_reviews,medical_condition_url,drug_link
551,3,chlordiazepoxide,anxiety,"anxiety other names: anxiety states; nerves; nervousness anxiety is an emotion characterized by feelings of tension, worried thoughts, and accompanied by physical symptoms such as sweating, trembling, voice changes, or increased blood pressure. it may also be called nervousness. occasional anxiety concerning a stressful or uncomfortable event is normal. however, if a person feels disproportionate levels of anxiety or it is present almost continuously, it might be diagnosed as an anxiety disorder. what causes anxiety? anxiety is part of our survival response and is the way our body responds to potentially harmful or worrying triggers. strong emotions or fear cause a surge of epinephrine (...",2%,rx,N,4,1,7.7,35.0,https://www.drugs.com/condition/anxiety.html,https://www.drugs.com/mtm/chlordiazepoxide.html


In [ ]:
from thirdai import bolt
import os

config_dir = "/content/drive/MyDrive/ThirdAI_Hackathon/Demos-main/configs"

model = bolt.UniversalDeepTransformer(
    data_types={
        "QUERY": bolt.types.text(),
        "DOC_ID": bolt.types.categorical(),
    },
    target="DOC_ID",
    n_target_classes=dataframe.shape[0],
    integer_target=True,
    model_config=os.path.join(config_dir, "embeddings_and_cold_start.config")
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model.cold_start(
    filename=sampled_catalog_file,
    strong_column_names=["medical_condition_description","medical_condition"],
    weak_column_names=["rx_otc","alcohol","rating"],
    learning_rate=0.0015,
    epochs=10,
    batch_size=10,
    metrics=["categorical_accuracy"],
)

loaded data | source '/content/drive/MyDrive/ThirdAI_Hackathon/Main Project Files/drugs_for_common_treatments-0.75.csv' | vectors 5938 | batches 594 | time 0s | complete

train | epoch 0 | train_steps 2107 | train_categorical_accuracy=0.146851  | train_batches 594 | time 47s

train | epoch 1 | train_steps 2701 | train_categorical_accuracy=0.289323  | train_batches 594 | time 46s

train | epoch 2 | train_steps 3295 | train_categorical_accuracy=0.387336  | train_batches 594 | time 51s

train | epoch 3 | train_steps 3889 | train_categorical_accuracy=0.423038  | train_batches 594 | time 46s

train | epoch 4 | train_steps 4483 | train_categorical_accuracy=0.4387  | train_batches 594 | time 46s

train | epoch 5 | train_steps 5077 | train_categorical_accuracy=0.449478  | train_batches 594 | time 51s

train | epoch 6 | train_steps 5671 | train_categorical_accuracy=0.460761  | train_batches 594 | time 46s

train | epoch 7 | train_steps 6265 | train_categorical_accuracy=0.456719  | train_batches

{'epoch_times': [47.0, 46.0, 51.0, 46.0, 46.0, 51.0, 46.0, 49.0, 46.0, 49.0],
 'train_categorical_accuracy': [0.1468507945537567,
  0.28932300209999084,
  0.38733580708503723,
  0.4230380654335022,
  0.4386999011039734,
  0.4494779407978058,
  0.4607611894607544,
  0.45671942830085754,
  0.45554059743881226,
  0.461266428232193]}

In [ ]:
model_save_path = './doctorg.model'

## save the model
model.save(model_save_path)



In [ ]:
## load it back
model = bolt.UniversalDeepTransformer.load(model_save_path)

In [ ]:
## Helper function to print the results
def top_k_drug(query, k):
    result = model.predict({"QUERY": query})
    #
    k = min(k, len(result) - 1)
    sorted_drug_names = result.argsort()[-k:][::-1]
    #
    for doc_id in sorted_drug_names:
        print(dict(dataframe.iloc[doc_id,[0,1,2]]))
        print('******************************')
    return sorted_drug_names


In [ ]:
## example 1
result=top_k_drug('cancer' , 5)  # pore-clogging, Blackheads,Vulgaris for acne pain skin care etc. Pimples bla bla

#Aids k liye --- HIV , Sex, Human immunodeficiency virus, sexually transmitted, babies bla bla

#High cholestrol k liye: Cholesteremia; Cholesterol, Elevated Levels; Cholesterol, High; Cholesterolemia; Familial Hypercholesterolemia;


{'DOC_ID': 2286, 'drug_name': 'ifosfamide', 'medical_condition': 'cancer'}
******************************
{'DOC_ID': 1945, 'drug_name': 'cyclophosphamide', 'medical_condition': 'cancer'}
******************************
{'DOC_ID': 154, 'drug_name': 'etopophos', 'medical_condition': 'cancer'}
******************************
{'DOC_ID': 2131, 'drug_name': 'topotecan', 'medical_condition': 'cancer'}
******************************
{'DOC_ID': 2030, 'drug_name': 'doxorubicin', 'medical_condition': 'cancer'}
******************************


In [ ]:
result=top_k_drug('acne' , 5)

{'DOC_ID': 1246, 'drug_name': 'excedrin tension headache', 'medical_condition': 'osteoarthritis'}
******************************
{'DOC_ID': 1318, 'drug_name': 'sastid', 'medical_condition': 'acne'}
******************************
{'DOC_ID': 2965, 'drug_name': 'capron dmt', 'medical_condition': 'hayfever'}
******************************
{'DOC_ID': 2774, 'drug_name': 'acetaminophen / chlorpheniramine / dextromethorphan / phenylephrine', 'medical_condition': 'colds & flu'}
******************************
{'DOC_ID': 93, 'drug_name': 'cope', 'medical_condition': 'pain'}
******************************


In [ ]:
print(result) #Top most prediction ID should be printed

[2825 2595 1941 1470 1372]


In [ ]:
drug_name=dataframe.iloc[result[0],1]
print(drug_name)

soriatane


In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=64d117d5c0ad1bb87ef58e93d3a3ff71d77fbd92fb1fb360d278375180514223
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
import wikipedia
summary = wikipedia.summary(drug_name, sentences=5)
sentences = summary.split('. ')
print("Information about the Drug :",drug_name)
for sentence in sentences:
    print(sentence + '.')

Information about the Drug : soriatane
Acitretin (trade names Soriatane and Neotigason) is a second-generation retinoid.
 It is taken orally, and is typically used for psoriasis.Acitretin is an oral retinoid used in the treatment of severe resistant psoriasis.
Because of the potential for problems and severe side effects it is generally used in only very severe cases of psoriasis that have been unresponsive to other treatments.
 It binds to nuclear receptors that regulates gene transcription.
 They induce keratinocyte differentiation and reduce epidermal hyperplasia, leading to the slowing of cell reproduction..
